# **Week 11: 협업 필터링 (Surprise 라이브러리)**

* Surprise 라이브러리는 추천 시스템을 쉽게 구현할 수 있도록 도와주는 Python 라이브러리로 주로 협업 필터링 기반의 추천 알고리즘을 지원
* 다양한 알고리즘과 평가 방법을 제공하며 추천 시스템 학습 및 실험에 매우 유용 
* 아래는 Surprise 라이브러리의 주요 기능과 사용 방법에 대한 구체적인 설명을 제공함

Surprise 라이브러리 공식 홈페이지: https://surpriselib.com/

In [27]:
%pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.


### **1. Surprise의 주요 기능**

* **데이터셋 관리**: Surprise는 사용자-아이템 평점 데이터를 손쉽게 로드하고 관리할 수 기능 제공하며, Dataset과 Reader 클래스를 사용하여 다양한 형식의 데이터를 쉽게 호출 가능
* **추천 알고리즘**: KNN 기반 협업 필터링, 행렬 분해 기반의 SVD, NMF 등 여러 종류의 추천 알고리즘을 지원
* **유사도 측정**: 사용자 또는 아이템 간의 유사도를 계산할 수 있도록 코사인 유사도, 피어슨 상관계수 등의 유사도 측정 방법을 지원
* **모델 평가**: accuracy 모듈을 통해 MAE, RMSE와 같은 평가지표를 간단히 계산할 수 있으며 교차 검증 기능도 제공

### **2. Surprise 주요 클래스와 사용 방법**

**(1)  데이터 준비 (Dataset과 Reader)**
 * Reader: 데이터 파일의 형식을 정의하는 클래스
    * 평점의 범위를 설정할 수 있으며 rating_scale 파라미터를 통해 사용자-아이템 평점 데이터의 범위를 지정
 * Dataset: 데이터를 로드하고 학습 및 테스트 데이터를 준비하는 클래스
    * load_from_df() 메서드를 사용하여 데이터프레임 형식의 데이터를 Surprise 형식으로 변환

In [35]:
import pandas as pd

In [49]:
ratings = pd.read_csv('ratings.dat', sep='::', engine='python', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'], encoding='latin1')
ratings

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [51]:
import pandas as pd
from surprise import Dataset, Reader

# 데이터 준비
ratings = pd.read_csv('ratings.dat', sep='::', engine='python', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'], encoding='latin1')
reader = Reader(rating_scale=(1, 5)) #1점부터 5점차이로 나눔
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader) #Surpeise 라이브러리를 사용하기 위해 형식으로 변환

In [54]:
 data

<surprise.dataset.DatasetAutoFolds at 0x231b1e9fad0>은 scikit-suprise라이브러리의 DataseAutoFolds객체가 정상적으로 객체가 생성되었다는 출력임

**(2) 학습 및 테스트 데이터 분할 (train_test_split)**
 * train_test_split 함수를 사용하여 학습 데이터와 테스트 데이터를 쉽게 분할
 *  Surprise 라이브러리의 train_test_split은 scikit-learn의 함수와 유사하지만, Surprise의 데이터 형식에 맞게 설계

 다양한 데이터 분할 함수는 홈페이지 참고: https://surprise.readthedocs.io/en/stable/model_selection.html

In [60]:
from surprise.model_selection import train_test_split 
# suprise라이브러리 내제된 함수 사용
# 학습 및 테스트 데이터로 분할
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

**(3) 추천 알고리즘 (KNNBasic, KNNWithMeans, KNNBaseline 등)**
 * KNNBasic: 사용자 또는 아이템 간 유사도를 기반으로 하는 기본 KNN 알고리즘
 * KNNWithMeans: 사용자 또는 아이템의 평균 평점을 고려하여 예측 정확도를 향상시킨 KNN 알고리즘
 * KNNWithZScore: 평점을 Z-점수로 표준화하여 평점의 분포를 반영하는 KNN 알고리즘
 * KNNBaseline: 사용자와 아이템의 baseline 평점을 보정하여 보다 정교한 예측을 수행하는 KNN 알고리즘

 다양한 추천 알고리즘은 홈페이지 참고: https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [65]:
from surprise import KNNBasic

# 사용자 기반 협업 필터링 설정 (피어슨 상관계수 사용)
sim_options = {
    'name': 'pearson',
    'user_based': True #사용자 기반
}

model = KNNBasic(k=20, sim_options=sim_options) # k: 이웃 사용자 수
model.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


**(4) 유사도 옵션 (sim_options)**
 * **sim_options**를 사용하여 유사도 계산 방식을 설정
    * name: 유사도 계산 방법을 지정 (예: 'cosine', 'pearson')
    * user_based: True이면 사용자 기반 유사도, False이면 아이템 기반 유사도를 사용

다양한 유사도 옵션은 홈페이지 참고: https://surprise.readthedocs.io/en/stable/similarities.html

In [67]:
sim_options = {
    'name': 'cosine',  # 코사인 유사도
    'user_based': False  # 아이템 기반 협업 필터링
}

**(5) 모델 평가 (accuracy)**
 * accuracy 모듈을 사용하여 모델의 예측 성능을 평가
 * MAE(Mean Absolute Error), RMSE(Root Mean Squared Error) 등 평가지표 지원

 다양한 평가지표는 홈페이지 참고: https://surprise.readthedocs.io/en/stable/accuracy.html

In [ ]:
from surprise import accuracy

# 테스트 데이터에 대한 평점 예측
predictions = model.test(testset)

# 성능 평가 (MAE, RMSE)
mae = accuracy.mae(predictions, verbose=True)
rmse = accuracy.rmse(predictions, verbose=True)

### **3. Surprise를 사용한 협업 필터링의 단계**

1. **데이터 준비:** 사용자-아이템 평점 데이터셋을 로드하고 Reader를 사용하여 Dataset을 생성
2. **학습 및 테스트 데이터 분할:** train_test_split을 사용하여 학습과 테스트 데이터를 분할
3. **모델 선택 및 학습:** 사용할 알고리즘(KNNBasic, KNNWithMeans 등)을 선택하고, 학습 데이터를 사용하여 모델을 학습
4. **유사도 설정:** 유사도 계산 방식(sim_options)을 설정
5. **모델 예측 및 평가:** 테스트 데이터를 사용하여 평점을 예측하고, accuracy를 통해 모델의 성능을 평가

### **4. Surprise의 장점**

* 간편한 사용성: 협업 필터링 알고리즘의 구현과 평가를 간단히 수행 가능
* 다양한 알고리즘 지원: KNN, SVD, NMF와 같은 다양한 추천 알고리즘을 기본적으로 지원
* 평가 도구: 성능 평가를 위한 다양한 평가지표와 교차 검증 도구를 제공

### **5. 주요 헙업 필터링 알고리즘 비교**

* KNNBasic: 단순히 이웃 기반의 평점을 평균하여 예측
* KNNWithMeans: 사용자/아이템의 평균을 보정하여 예측 성능을 향상
* KNNWithZScore: 평점을 Z-점수로 변환하여 데이터의 편향 감소
* KNNBaseline: 사용자와 아이템의 baseline 편향을 보정하여 정확한 평점을 예측

### **6. Surprise 라이브러리 실습**

#### Step 1: 라이브러리 임포트

In [ ]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, accuracy
from surprise.model_selection import train_test_split as surprise_train_test_split

#### Step 2: MovieLens 1M 데이터셋 불러오기

In [ ]:
ratings = pd.read_csv('ratings.dat', sep='::', engine='python', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'], encoding='latin1')

In [ ]:
ratings.head()

#### Step 3: Surprise 라이브러리용 데이터 준비

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

#### Step 4: 학습 데이터와 테스트 데이터로 분할 (80:20 비율)

In [ ]:
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)

#### Step 5: 사용자 기반 협업 필터링 (피어슨 상관계수)

In [ ]:
sim_options_user_pearson = {
    'name': 'pearson',
    'user_based': True  # 사용자 기반 협업 필터링
}

knn_models = {
    'KNNBasic': KNNBasic,  # 기본 KNN 알고리즘
    'KNNWithMeans': KNNWithMeans,  # 평균 평점을 고려한 KNN 알고리즘
    'KNNWithZScore': KNNWithZScore,  # 평점을 표준화하여 예측하는 KNN 알고리즘
    'KNNBaseline': KNNBaseline  # Baseline 평점을 기반으로 예측하는 KNN 알고리즘
}

In [ ]:
for model_name, model_class in knn_models.items():
    print(f"\n--- {model_name} (사용자 기반, 피어슨 상관계수) ---")
    model = model_class(k= 40, sim_options=sim_options_user_pearson)
    model.fit(trainset)

    # 테스트 데이터에 대한 예측
    predictions = model.test(testset)

    # 모델 성능 평가 (MAE 및 RMSE)
    mae = accuracy.mae(predictions, verbose=True)
    rmse = accuracy.rmse(predictions, verbose=True)

    # 평가 결과 출력
    print(f'{model_name} (사용자 기반, 피어슨 상관계수) MAE: {mae}')
    print(f'{model_name} (사용자 기반, 피어슨 상관계수) RMSE: {rmse}')

#### Step 6: 아이템 기반 협업 필터링 (코사인 유사도)

In [ ]:
sim_options_item_cosine = {
    'name': 'cosine',
    'user_based': False  # 아이템 기반 협업 필터링
}

In [ ]:
for model_name, model_class in knn_models.items():
    print(f"\n--- {model_name} (아이템 기반, 코사인 유사도) ---")
    model = model_class(sim_options=sim_options_item_cosine)
    model.fit(trainset)

    # 테스트 데이터에 대한 예측
    predictions = model.test(testset)

    # 모델 성능 평가 (MAE 및 RMSE)
    mae = accuracy.mae(predictions, verbose=True)
    rmse = accuracy.rmse(predictions, verbose=True)

    # 평가 결과 출력
    print(f'{model_name} (아이템 기반, 코사인 유사도) MAE: {mae}')
    print(f'{model_name} (아이템 기반, 코사인 유사도) RMSE: {rmse}')

#### Step 7: 다양한 이웃 크기를 고려한 아이템 기반 협업 필터링

In [ ]:
neighbor_sizes = [10, 20, 40]  # 다양한 이웃 크기 설정

In [ ]:
for k in neighbor_sizes:
    for model_name, model_class in knn_models.items():
        print(f"\n--- {model_name} (아이템 기반, 코사인 유사도, k={k}) ---")
        model = model_class(k=k, sim_options=sim_options_item_cosine)
        model.fit(trainset)

        # 테스트 데이터에 대한 예측
        predictions = model.test(testset)

        # 모델 성능 평가 (MAE 및 RMSE)
        mae = accuracy.mae(predictions, verbose=True)
        rmse = accuracy.rmse(predictions, verbose=True)

        # 평가 결과 출력
        print(f'{model_name} (아이템 기반, 코사인 유사도, k={k}) MAE: {mae}')
        print(f'{model_name} (아이템 기반, 코사인 유사도, k={k}) RMSE: {rmse}')